In [1]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

In [2]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv()

True

In [3]:
# 定义期望的数据结构
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # 添加自定义验证逻辑
    @validator('setup')
    def question_ends_with_question_mark(cls, value: str) -> str:
        if not value.endswith('?'):
            raise ValueError("Badly formed question! It should end with a question mark.")
        return value

# 创建解析器实例
parser = PydanticOutputParser(pydantic_object=Joke)

# 创建提示模板
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 创建模型实例
llm = OpenAI(temperature = 0.0)

# 使用LCEL调用模型并解析输出
chain = prompt | llm | parser
chain.invoke("Tell me a joke.")

/var/folders/rq/kthvngcn3r769fzgn106ymd00000gn/T/ipykernel_35694/2000046976.py:7: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator('setup')


Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')